In [1]:
import pandas as pd
import numpy as np

from requests_html import HTMLSession
import random
from time import sleep

/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
session = HTMLSession()

In [3]:
def scrap_page(page_number):
    df = pd.DataFrame(columns = ['marque_modele', 'version', 'energie', 'boite_vitesse', 'puis_fisc', 'date', 'malus', 'prix'])
    s = session.get(url='https://www.autoplus.fr/prix-neuf/?page=%i#advanced-filters'%page_number)
    for i in range(1, 32):
        elements_list = s.html.xpath('//*[@id="content"]/div[6]/table/tbody/tr[%i]'%i)[0].text.split('\n')
        elements_list = [elem.replace(u'\xa0', '').replace('€', '') for elem in elements_list]
        
        try:
            df.loc[i] = elements_list
            
        except ValueError:
            pass
            
    return df

def scrap_pages(number_pages):
    df_final = pd.DataFrame()
    
    for i in range(1, number_pages):
        sleep(random.randint(1, 10))
        
        df_ = scrap_page(i)
        
        df_final = pd.concat([df_final, df_])
        
    return df_final

In [4]:
df_prix = scrap_pages(475)

In [5]:
df_prix.shape

(12067, 8)

In [6]:
df_prix.to_csv(r'car_price.csv', index = False)